In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import urllib
import zipfile
import seaborn as sns

import torch
dtype = torch.float
device = torch.device("cuda:0") 
from torch.distributions import normal
import matplotlib.pyplot as plt, numpy as np, pickle
from scipy.stats import norm
from tifffile import imread
import sys
sys.path.append('/home/ashesh.ashesh/code/PPN2V')
from src.ppn2v.pn2v import *
import src.ppn2v.pn2v.gaussianMixtureNoiseModel
import src.ppn2v.pn2v.histNoiseModel
import src.ppn2v.pn2v.prediction
from src.ppn2v.pn2v.utils import *
from src.scripts.train_n2v import get_bestmodelname, load_data
from disentangle.core.psnr import RangeInvariantPsnr



In [ ]:
!ls /home/ashesh.ashesh/training/noise_model/2307/37/

In [ ]:
import glob
def gmm_model(model_directory):
    for fpath in glob.glob(f'{model_directory}/*'):
        if 'GMMNoiseModel' in fpath:
            return fpath
            

In [ ]:
model_directory = '/home/ashesh.ashesh/training/noise_model/2307/25'
model_fpath = gmm_model(model_directory)

data_dir = '/group/jug/ashesh/data/ventura_gigascience/'
# data_dir = '/group/jug/ashesh/data/BSD68_reproducibility_data/val/'
data_dir = data_dir[:-1] if '/' == data_dir[-1] else data_dir

dset = 'ventura_gigascience'
low_snr_filename = 'actin-60x-noise2-lowsnr.tif'
high_snr_filename = 'actin-60x-noise2-highsnr.tif'

# low_snr_filename =  'DCNN400_validation_gaussian25.npy'
# high_snr_filename = 'DCNN400_validation_gaussian25.npy'

pred_actin = np.load('/home/ashesh.ashesh/PhDNotebooks/disentangle/NoiseModel/split_predictions_ch1.npy')
pred_mito = np.load('/home/ashesh.ashesh/PhDNotebooks/disentangle/NoiseModel/split_predictions_ch2.npy')

datatype = low_snr_filename.split('-')[0]
if datatype == 'actin':
    pred_data = pred_actin
elif datatype == 'mito':
    pred_data = pred_mito
    
low_snr_data = load_data(os.path.join(data_dir, low_snr_filename))
high_snr_data = load_data(os.path.join(data_dir, high_snr_filename))


In [ ]:
!ls -lhrt $model_fpath

In [ ]:
from disentangle.nets.gmm_noise_model import GaussianMixtureNoiseModel
model = GaussianMixtureNoiseModel(params=np.load(model_fpath))


In [ ]:
from tqdm import tqdm 
results=[]
meanRes=[]
resultImgs=[]
inputImgs=[]
dataTest = pred_data#low_snr_data

for index in tqdm(range(dataTest.shape[0])):

    im=dataTest[index]
    # We are using tiling to fit the image into memory
    # If you get an error try a smaller patch size (ps)
    means = src.ppn2v.pn2v.prediction.tiledPredict(im, net, ps=256, overlap=48,
                                            device=device, noiseModel=None)
    resultImgs.append(means)
    inputImgs.append(im)
    
prediction = np.concatenate([x[None] for x in resultImgs],axis=0).astype(np.int32)

In [ ]:
_,ax = plt.subplots(figsize=(18,6),ncols=3)
img_idx = 0 #np.random.randint(0,high=len(inputImgs))
hs = 600
ws = 600
hN = 800
wN = 800
ax[0].imshow(inputImgs[img_idx][hs:hs+hN, ws:ws+wN])
ax[1].imshow(resultImgs[img_idx][hs:hs+hN, ws:ws+wN])
ax[2].imshow(high_snr_data[img_idx][hs:hs+hN, ws:ws+wN])

In [ ]:
_ = plt.hist(inputImgs[img_idx].reshape(-1,),bins=50, color='red')
_ = plt.hist(resultImgs[img_idx].reshape(-1,),bins=50, color='blue')

In [ ]:
np.save(f'/home/ashesh.ashesh/PhDNotebooks/disentangle/NoiseModel/split_predictions_denoised_{datatype}.npy', np.array(resultImgs))

In [ ]:
psnrs = RangeInvariantPsnr(high_snr_data.astype(np.int32), prediction)
print(f'{torch.mean(psnrs).item():.1f}','+-', f'{torch.std(psnrs).item():.1f}')